# Projeto COVID-19

## J.G

In [1]:
# importando bibliotecas

import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import re
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from fbprophet import prophet

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
# importando dados para o projeto

url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'

In [ ]:
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

In [ ]:
# Função para arrumar as colunas

def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
# Colunas arrumadas

df.columns = [corrige_colunas(col) for col in df.columns]
df

# Brasil

Selecionando dados do Brasil para investigar.

In [ ]:
df.loc[df['countryregion'] == 'Brazil']

In [ ]:
df_brasil = df.loc[
    (df.confirmed > 0) &
    (df.countryregion == 'Brazil')
]

df_brasil


# Casos confirmados


In [ ]:
# Gráfico da evolução de casos confirmados

px.line(df_brasil, x='observationdate', y='confirmed', title='Casos confirmados no Brasil')

# Novos casos por dia


In [ ]:
# Técnica de programação funcional

df_brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else df_brasil['confirmed'].iloc[x] - df_brasil['confirmed'].iloc[x-1],
    np.arange(df_brasil.shape[0])
))

In [ ]:
df_brasil

In [ ]:
# Visualizando

px.line(df_brasil, 'observationdate', 'novoscasos', title='Novos casos por dia no Brasil')

# Mortes


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_brasil.observationdate, y=df_brasil.deaths, name='Mortes',
              mode='lines+markers', line={'color':'darkred'})
)

# Layout

fig.update_layout(title='Mortes por covid-19 no Brasil')
fig.show()

# Taxa de crescimento

In [ ]:
def taxa_crescimento(data, var, data_inicio=None, data_fim=None):
    #Se data início for None, define como a primeira data disponível
    
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[var] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    if data_fim == None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
        
    # Define os valores do presente e passado
    
    passado = data.loc[data.observationdate == data_inicio, var].values[0]
    presente = data.loc[data.observationdate == data_fim, var].values[0]
    
    # Define o número de pontos no tempo que vamos analisar
    
    n = (data_fim - data_inicio).days
    
    # Calcular a taxa
    
    taxa = (presente/passado)**(1/n) - 1
    
    return taxa*100

In [ ]:
# Taxa de crescimento do covid-19 médio no Brasil

taxa_crescimento(df_brasil, 'confirmed')

In [ ]:
def taxa_crescimento_diaria(data, var, data_inicio=None):

    #Se data início for None, define como a primeira data disponível
    
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[var] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
    
    data_fim = data.observationdate.max()
    n = (data_fim - data_inicio).days
    
    # Taxa calculada de um dia para o outro
    
    taxas = list(map(
        lambda x: (data[var].iloc[x] - data[var].iloc[x-1]) / data[var].iloc[x-1],
        range(1, n+1)
    ))
    
    return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diaria(df_brasil, 'confirmed')


In [ ]:
tx_dia

In [ ]:
primeiro_dia = df_brasil.observationdate.loc[df_brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, df_brasil.observationdate.max())[1:],
       y = tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

# Predições


In [ ]:
confirmados = df_brasil.confirmed
confirmados.index = df_brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

# Arima


In [ ]:
modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name='Observados'
))

fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), name='Predict'
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20', '2020-06-20'), y=modelo.predict(31), name='Forecast'
))

fig.update_layout(title='Previsão de casos confirmados no Brasil nos próximos 30 dias')



# Modelo de crescimento


In [ ]:
# Preprocessamento

train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

# Renomeando colunas

train.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)
test.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)

#Definir o modelo de crescimento

profeta = Prophet(growth='logistic', changepoints=['2020-03-21', '2020-03-30', '2020-04-25',
                                                   '2020-05-03', '2020-05-10'])

pop = 211463256
train['cap'] = pop

# Treina o modelo
profeta.fit(train)

# Construir previsões para o futuro

future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop
forecast = profeta.predict(future_dates)


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()